Import all Library

In [1]:
import re
import string
import pandas as pd
import csv
import requests

from io import StringIO
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Dense, Dropout, Input, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, classification_report
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords

# download stopwords dan punctuation dari library nltk
import nltk
nltk.download('stopwords')
nltk.download('punkt')

# download mpstemmer dari github ariaghora
%pip install python-Levenshtein
%pip install --upgrade git+https://github.com/ariaghora/mpstemmer.git
from mpstemmer import MPStemmer

[nltk_data] Error loading stopwords: <urlopen error [Errno 11001]
[nltk_data]     getaddrinfo failed>
[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     UNEXPECTED_EOF_WHILE_READING] EOF occurred in
[nltk_data]     violation of protocol (_ssl.c:1006)>


Note: you may need to restart the kernel to use updated packages.
  Cloning https://github.com/ariaghora/mpstemmer.git to c:\users\vega\appdata\local\temp\pip-req-build-dn027jdd
Note: you may need to restart the kernel to use updated packages.


  Running command git clone --filter=blob:none --quiet https://github.com/ariaghora/mpstemmer.git 'C:\Users\vega\AppData\Local\Temp\pip-req-build-dn027jdd'
  fatal: unable to access 'https://github.com/ariaghora/mpstemmer.git/': Could not resolve host: github.com
  error: subprocess-exited-with-error
  
  × git clone --filter=blob:none --quiet https://github.com/ariaghora/mpstemmer.git 'C:\Users\vega\AppData\Local\Temp\pip-req-build-dn027jdd' did not run successfully.
  │ exit code: 128
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
error: subprocess-exited-with-error

× git clone --filter=blob:none --quiet https://github.com/ariaghora/mpstemmer.git 'C:\Users\vega\AppData\Local\Temp\pip-req-build-dn027jdd' did not run successfully.
│ exit code: 128
╰─> See above for output.

note: This error originates from a subprocess, and is likely not a problem with pip.


In [2]:
%pip freeze > requirements.txt

Note: you may need to restart the kernel to use updated packages.


Loading Dataset

In [3]:
# import variabel dari modul scraping
from PS_Gojek import app_reviews_df

# menghapus isi konten yang kosong dan duplikat
gojek_clean_df = app_reviews_df.dropna()
gojek_clean_df = gojek_clean_df.drop_duplicates()
jumlah_ulasan_setelah_hapus_duplikat, jumlah_kolom_setelah_hapus_duplikat = gojek_clean_df.shape
gojek_clean_df.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion
0,ad5d42d7-26b3-4b05-84d5-ec7a1630195c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Opsi pemilihan GOFOOD ""terdekat"" sekarang kura...",2,137,4.89.1,2024-06-26 03:44:12,"Hai Kak Christian, mohon maaf atas ketidaknyam...",2024-06-26 09:25:34,4.89.1
1,fcecd263-6061-4de6-b9f7-ecc2608922dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi sudah sangat baik, tapi saya kecewa, ...",1,57,4.90.2,2024-07-11 11:37:32,"Hai Kak Boy, mohon maaf atas ketidaknyamananny...",2024-07-11 13:44:23,4.90.2
2,1bf37a40-af8c-4d1f-a677-07a46b121241,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Dear gojek..mengapa setelah pembaruan aplikasi...,1,66,4.90.2,2024-07-17 11:33:53,"Hai David Sinaga, mohon maaf ya. Laporan yang ...",2023-07-11 12:44:39,4.90.2
3,a44b1111-44c4-43c6-8c62-828db2c92fe8,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,aplikasinya bosok. Saya order mobil kebaca di ...,1,1,4.90.2,2024-07-18 10:46:23,"Hai Kak @Maha Karma, mohon maaf atas ketidakny...",2024-07-18 13:13:06,4.90.2
4,e24b3821-bd15-4ff4-a751-8c221bac3a26,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,Saya mau order go car susahnya minta ampun. Pa...,1,128,4.89.1,2024-06-21 10:04:58,"Mohon maaf atas kendala yang dialami, Kak Auli...",2024-06-21 11:26:46,4.89.1


Pre-Proseccing

In [4]:
# data slangwords
slangwords = {
    'otw': 'on the way',
    'pas': 'tepat',
    'kzl': 'kesal',
    'brg': 'barang',
    'dtg': 'datang',
    'sygn': 'sayang',
    'hrs': 'harus',
    'tp': 'tapi',
    'kmrn': 'kemarin',
    'kmrn': 'kemarin',
    'dpt': 'dapat',
    'bgs': 'bagus',
    'tdk': 'tidak',
    'brp': 'berapa',
    'gw': 'saya',
    'sis': 'sisa',
    'bkn': 'bukan',
    'kalo': 'kalau',
    'tdk': 'tidak',
    'blm': 'belum',
    'bnyk': 'banyak',
    'sih': 'sih',
    'jg': 'juga',
    'km': 'kilometer',
    'gmn': 'gimana',
    'bro': 'saudara',
    'sis': 'saudari',
    'asap': 'secepat mungkin',
    'cpt': 'cepat',
    'dr': 'dari',
    'kli': 'kali',
    'emg': 'memang',
    'udh': 'sudah',
    'plg': 'pulang',
    'drmn': 'dari mana',
    'aneh': 'aneh',
    'malah': 'malah',
    'jadi': 'jadi',
    'aja': 'saja',
    'dlu': 'dulu',
    'sinyal': 'signal',
    'ngecharge': 'mengisi daya',
    'cr': 'cari',
    'bisa': 'bisa',
    'lho': 'loh',
    'yah': 'yah',
    'banget': 'sekali',
    'loh': 'loh',
    'mlm': 'malam',
    'pagi': 'pagi',
    'siang': 'siang',
    'sore': 'sore',
    'trs': 'terus',
    'sampe': 'sampai',
    'skrg': 'sekarang',
    'btw': 'by the way',
    'punya': 'punya',
    'km': 'kamu',
    'temen': 'teman',
    'dong': 'dong',
    'pdhl': 'padahal',
    'tlp': 'telepon',
    'lg': 'lagi',
    'udh': 'sudah',
    'klr': 'keluar',
    'wkt': 'waktu',
    'dmn': 'di mana',
    'antri': 'antre',
    'lama': 'lama',
    'hrg': 'harga',
    'apk': 'aplikasi',
    'ga': 'tidak',
    'tau': 'tahu',
    'susah': 'susah',
    'mudah': 'mudah',
    'error': 'error',
    'admin': 'admin',
    'cs': 'customer service',
    'top': 'terbaik',
    'mantap': 'hebat',
    'kurir': 'pengantar',
    'tarif': 'tarif',
    'transaksi': 'transaksi',
    'bengkel': 'bengkel',
    'bus': 'bus',
    'mampir': 'singgah',
    'sopir': 'pengemudi',
    'taksi': 'taksi',
    'ojol': 'ojek online',
    'angkot': 'angkutan kota',
    'bensin': 'bahan bakar',
    'ngebut': 'cepat',
    'ngetem': 'menunggu penumpang',
    'pelit': 'kikir',
    'service': 'layanan',
    'promo': 'promosi',
    'cod': 'cash on delivery',
    'respon': 'tanggapan',
    'feedback': 'umpan balik',
    'refund': 'pengembalian dana',
    'voucher': 'kupon',
    'diskon': 'potongan harga',
    'murah': 'murah',
    'mahal': 'mahal',
    'macet': 'macet',
    'jemput': 'menjemput',
    'antar': 'mengantar',
    'tujuan': 'destinasi',
    'bonus': 'bonus',
    'batal': 'membatalkan',
    'delay': 'penundaan',
    'reschedule': 'menjadwal ulang',
    'rating': 'penilaian',
    'review': 'ulasan',
    'komentar': 'komentar',
    'kualitas': 'kualitas',
    'kecewa': 'kecewa',
    'puas': 'puas',
    'rahasia': 'privasi',
    'update': 'pembaruan',
    'lapor': 'melaporkan',
    'refund': 'pengembalian dana',
    'klaim': 'klaim',
    'saldo': 'saldo',
    'rekap': 'rekapitulasi',
    'driver': 'pengemudi',
    'partner': 'mitra',
    'fasilitas': 'fasilitas',
    'kendala': 'hambatan',
    'jemputan': 'penjemputan',
    'tujuan': 'destinasi',
    'order': 'pesanan',
    'pesan': 'memesan',
    'bayar': 'membayar',
    'aman': 'aman',
    'amanah': 'tepercaya',
    'layanan': 'layanan',
    'cepat': 'cepat',
    'praktis': 'praktis',
    'ribet': 'rumit',
    'keluhan': 'keluhan',
    'kelola': 'mengelola',
    'promo': 'promosi',
    'frekuensi': 'jumlah',
    'stabil': 'stabil',
    'akurat': 'tepat',
    'jadwal': 'jadwal',
    'estimasi': 'perkiraan',
    'pengalaman': 'pengalaman',
    'memuaskan': 'memuaskan',
    'klaim': 'klaim',
    'rating': 'penilaian',
    'negatif': 'negatif',
    'positif': 'positif',
    'review': 'ulasan',
    'pembayaran': 'pembayaran',
    'transaksi': 'transaksi',
    'konfirmasi': 'konfirmasi',
    'notifikasi': 'pemberitahuan',
    'user': 'pengguna',
    'komplain': 'keluhan',
    'terima': 'menerima',
    'kirim': 'mengirim',
    'cetak': 'mencetak',
    'logistik': 'pengiriman',
    'kontak': 'menghubungi',
    'resi': 'nomor pengiriman',
    'pelanggan': 'pelanggan',
    'transparan': 'terbuka',
    'penyelesaian': 'solusi',
    'garansi': 'jaminan',
    'pelayanan': 'layanan',
    'responsif': 'cepat tanggap',
    'respons': 'tanggapan',
    'integritas': 'kejujuran',
    'kecepatan': 'kecepatan',
    'pengembalian': 'pengembalian',
    'kompetitif': 'bersaing',
    'kemudahan': 'kemudahan',
    'fleksibel': 'mudah',
    'tersedia': 'tersedia',
    'sesuai': 'cocok',
    'menyenangkan': 'menyenangkan',
    'kekurangan': 'kekurangan',
    'keuntungan': 'keuntungan',
    'kerugian': 'kerugian',
    'keperluan': 'keperluan'
    }

# mengubah kata-kata slang menjadi lebih baku dan mudah dipahami
def slangwordsText(text):
    words = text.split()
    words_fixed = [slangwords[word.lower()] if word.lower() in slangwords else word for word in words]

    return ' '.join(words_fixed)

# unicode untuk emoticon, emoji, simbol dan lain - lain
def emojiRemove(text):
    emoji_pattern = re.compile(
        "["
        "\U0001F600-\U0001F64F"
        "\U0001F300-\U0001F5FF"
        "\U0001F680-\U0001F6FF"
        "\U0001F700-\U0001F77F"
        "\U0001F780-\U0001F7FF"
        "\U0001F800-\U0001F8FF"
        "\U0001F900-\U0001F9FF"
        "\U0001FA00-\U0001FA6F"
        "\U0001FA70-\U0001FAFF"
        "\U00002702-\U000027B0"
        "\U000024C2-\U0001F251"
        "]+", flags=re.UNICODE
    )

    text = emoji_pattern.sub(r'', text)

    return text

# menghilangkan karakter spesial, whitespace dan non-whitespace
def textCleaning(text):
    text = re.sub(r'[0-9]+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub(r'#[A-Za-z0-9]+', '', text)
    text = re.sub(r'@[A-Za-z0-9]+', '', text)
    text = re.sub(r"http\S+", '', text)
    text = re.sub(r'RT[\s]', '', text)
    text = text.replace('\n', ' ')
    text = text.translate(str.maketrans('', '', string.punctuation))
    return text.strip()

# mengubah semua text menjadi huruf kecil
def caseFolding(text):
    return text.lower()

# pemberian token pada setiap text
def textTokenizing(text):
    text = word_tokenize(text)
    return text

# penyaringan text untuk menghilangkan kata-kata umum yang tidak diperlukan
def textFiltering(tokens):
    stopwords_indonesia = set(stopwords.words('indonesian'))
    stopwords_english = set(stopwords.words('english'))
    stopwords_indonesia.update(stopwords_english)
    return [word for word in tokens if word not in stopwords_indonesia]

# menghilangkan awalan dan akhiran kata dalam bahasa indonesia dan inggris
def textStemming(text):
    stemmer = MPStemmer()
    return ' '.join(stemmer.stem(word) for word in text.split())

# menggabungkan kata-kata menjadi sebuah kalimat
def toSentence(words):
    sentences = ' '.join(word for word in words)

    return sentences

# fungsi gabungan dari semua fungsi pre-processing diatas 
# kecuali toSentence untuk melihat hasil score polarity

def combineFunction(text):
    text = textCleaning(text)
    text = emojiRemove(text)
    text = caseFolding(text)
    text = slangwordsText(text)
    text = textStemming(text)
    tokens = textTokenizing(text)
    tokens = textFiltering(tokens)
    
    return tokens

In [5]:
# menggunakan fungsi kombinasi sampai textfiltering untuk melihat score polarity
gojek_clean_df['filtering_text'] = gojek_clean_df['content'].apply(combineFunction)

# kemudian diteruskan pada fungsi untuk digabungkan menjadi kalimat
gojek_clean_df['content_clean'] = gojek_clean_df['filtering_text'].apply(toSentence)

# menampilkan panjang baris data
print(len(gojek_clean_df))
gojek_clean_df.head(2)

29357


,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt,appVersion,filtering_text,content_clean
0,ad5d42d7-26b3-4b05-84d5-ec7a1630195c,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Opsi pemilihan GOFOOD ""terdekat"" sekarang kura...",2,137,4.89.1,2024-06-26 03:44:12,"Hai Kak Christian, mohon maaf atas ketidaknyam...",2024-06-26 09:25:34,4.89.1,"[opsi, pilih, gofood, efisien, pilih, opsi, hi...",opsi pilih gofood efisien pilih opsi hilang ga...
1,fcecd263-6061-4de6-b9f7-ecc2608922dc,Pengguna Google,https://play-lh.googleusercontent.com/EGemoI2N...,"Aplikasi sudah sangat baik, tapi saya kecewa, ...",1,57,4.90.2,2024-07-11 11:37:32,"Hai Kak Boy, mohon maaf atas ketidaknyamananny...",2024-07-11 13:44:23,4.90.2,"[aplikasi, kecewa, gojek, data, riwayat, trans...",aplikasi kecewa gojek data riwayat transaksi l...


Pemberian Label

In [8]:
# kamus untuk kata-kata positif
positive_dictionary = dict()

# import kamus positif dari github, credit to the owner: angelmetanosaa
res = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_positive.csv')

# logika jika data tidak dapat diambil
if res.status_code == 200:
    reader = csv.reader(StringIO(res.text), delimiter=',')
    
    # menambahkan kata-kata positif dan skor ke dalam positive_dictionary
    for row in reader:
        positive_dictionary[row[0]] = int(row[1])

else:
    print('Unable to fetch positive lexicon data')
    
# kamus untuk kata-kata negatif
negative_dictionary = dict()

# import kamus negatif dari github, credit to the owner: angelmetanosaa
res = requests.get('https://raw.githubusercontent.com/angelmetanosaa/dataset/main/lexicon_negative.csv')

# logika jika data tidak dapat diambil
if res.status_code == 200:
    reader = csv.reader(StringIO(res.text), delimiter=',')
    
    # menambahkan kata-kata negatif dan skor ke dalam negative_dictionary
    for row in reader:
        negative_dictionary[row[0]] = int(row[1])

else:
    print('Unable to fetch negative lexicon data')

In [9]:
# Fungsi untuk menentukan polaritas sentimen dari data review shopee
def analysis_sentiment(words):
    score = 0;
    
    # loop untuk mencari setiap kata pada setiap kalimat
    for word in words:
        
        # jika kata tersebut terdapat di dalam positive_dictionary maka score akan ditambah
        # sesuai dengan nilai pada positive_dictionary
        if(word in positive_dictionary):
            score = score + positive_dictionary[word]
        
        # jika kata tersebut terdapat di dalam negative_dictionary maka score akan ditambah
        # sesuai dengan nilai pada negative_dictionary
        if(word in negative_dictionary):
            score = score + negative_dictionary[word]
    
    # deklarasi variabel polarity untuk sebagai wadah nilai pada score
    polarity = 0
    
    # logika score untuk menentukan nilai polarity
    if (score > 0):
        polarity = 2 # Positive
    elif(score < 0 ):
        polarity = 0 # Negative
    else:
        polarity = 1 # Netral
        
    return polarity, score

In [10]:
# menganalisis pada kolom filtering_text
result = gojek_clean_df['filtering_text'].apply(analysis_sentiment)
result = list(zip(*result))

# menampilkan 
gojek_clean_df['polarity'] = result[0]
gojek_clean_df['polarity_score'] = result[1]
print(gojek_clean_df['polarity'].value_counts())
print('\n')
print(gojek_clean_df['content_clean'].head())

polarity
0    20119
2     7294
1     1944
Name: count, dtype: int64


0    opsi pilih gofood efisien pilih opsi hilang ga...
1    aplikasi kecewa gojek data riwayat transaksi l...
2    dear gojekmengapa aplikasi posisi kemudi titik...
3    aplikasi bosok pesan mobil baca map jalan canc...
4    pesan go car susah ampun jaring stabil berkali...
Name: content_clean, dtype: object


Data Splitting

In [11]:
# membagi data konten dan fitur
X = gojek_clean_df['content_clean']
Y = gojek_clean_df['polarity']

# memberikan nilai setiap fitur dengan teknik TF-IDF
tf_idf = TfidfVectorizer(max_features=1000, min_df=17, max_df=0.8)
X_tf_idf = tf_idf.fit_transform(X)

# mengubah vektor menjadi Dataframe
features_df = pd.DataFrame(X_tf_idf.toarray(), columns=tf_idf.get_feature_names_out())
features_df

,abang,abis,acc,aco,ad,admin,adu,aduh,agustus,ah,...,way,wifi,wilayah,wkwk,woi,xp,yaa,yaaa,yah,youtube
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.279123,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29352,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29353,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29354,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29355,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# membagi data latih dan uji dengan fungsi train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_tf_idf.toarray(), Y, test_size=0.2, random_state=42)

print('Train Data Shape: ', X_train.shape, y_train.shape)
print('Test Data Shape: ', X_test.shape, y_test.shape)

Train Data Shape:  (23485, 1000) (23485,)
Test Data Shape:  (5872, 1000) (5872,)


Model Mechine Learning menggunakan arsitektur RNN 3 macam variasi

Callbacks

In [13]:
# Callback untuk menghentikan training jika tidak ada peningkatan
early_stopping = EarlyStopping(
    restore_best_weights=True,
    patience=3,
    min_delta=0.01,
    monitor='val_loss'
)

In [14]:
# Callback untuk mengurangi learning rate jika validation loss tidak membaik dalam beberapa epoch
reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=0.0001
)

Model 1

In [15]:
# Model 1 menggunakan arsitektur RNN
Model_1 = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(1080, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(512, activation='relu'),
    Dropout(0.2),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.4),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Mengatur optimizer, loss dan metrics yang akan di tampilkan
Model_1.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

# Melatih Model RNN
hist_accuracy_1 = Model_1.fit(
    X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping, reduce_lr]
)

# Memprediksi model dengan data latih dan uji
y_pred_train_1 = Model_1.predict(X_train)
y_pred_test_1 = Model_1.predict(X_test)

# Konversi prediksi ke label
y_pred_train_labels_1 = y_pred_train_1.argmax(axis=1)
y_pred_test_labels_1 = y_pred_test_1.argmax(axis=1)

# Menghitung akurasi skor berdasarkan nilai prediksi data latih dan uji
accuracy_train_1 = accuracy_score(y_train, y_pred_train_labels_1)
accuracy_test_1 = accuracy_score(y_test, y_pred_test_labels_1)

# Menampilkan hasil akurasi 
print('\n\nTrain Accuracy: ', accuracy_train_1)
print('Test Accuracy: ', accuracy_test_1)
print('\nClassification Report:\n', classification_report(y_test, y_pred_test_labels_1))

Epoch 1/10
588/588 ━━━━━━━━━━━━━━━━━━━━ 35s 38ms/step - accuracy: 0.6093 - loss: 0.8893 - val_accuracy: 0.8258 - val_loss: 0.4442 - learning_rate: 0.0010
Epoch 2/10
588/588 ━━━━━━━━━━━━━━━━━━━━ 20s 34ms/step - accuracy: 0.8465 - loss: 0.4274 - val_accuracy: 0.8599 - val_loss: 0.3620 - learning_rate: 0.0010
Epoch 3/10
588/588 ━━━━━━━━━━━━━━━━━━━━ 21s 36ms/step - accuracy: 0.8754 - loss: 0.3477 - val_accuracy: 0.8680 - val_loss: 0.3424 - learning_rate: 0.0010
Epoch 4/10
588/588 ━━━━━━━━━━━━━━━━━━━━ 22s 37ms/step - accuracy: 0.8888 - loss: 0.3021 - val_accuracy: 0.8684 - val_loss: 0.3497 - learning_rate: 0.0010
Epoch 5/10
588/588 ━━━━━━━━━━━━━━━━━━━━ 21s 35ms/step - accuracy: 0.9016 - loss: 0.2709 - val_accuracy: 0.8648 - val_loss: 0.3839 - learning_rate: 0.0010
Epoch 6/10
588/588 ━━━━━━━━━━━━━━━━━━━━ 22s 38ms/step - accuracy: 0.9130 - loss: 0.2477 - val_accuracy: 0.8689 - val_loss: 0.3611 - learning_rate: 0.0010
734/734 ━━━━━━━━━━━━━━━━━━━━ 6s 7ms/step
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms

c:\Users\vega\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vega\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\vega\anaconda3\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Model 2

In [16]:
# Model 2 menggunakan arsitektur RNN
Model_2 = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(512, activation='relu'),
    BatchNormalization(),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    BatchNormalization(),
    Dropout(0.4),
    Dense(64, activation='relu'),
    BatchNormalization(),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Mengatur optimizer, loss dan metrics yang akan di tampilkan
Model_2.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

# Melatih Model RNN
hist_accuracy_2 = Model_2.fit(
    X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping, reduce_lr]
)

# Memprediksi model dengan data latih dan uji
y_pred_train_2 = Model_2.predict(X_train)
y_pred_test_2 = Model_2.predict(X_test)

# Konversi prediksi ke label
y_pred_train_labels_2 = y_pred_train_2.argmax(axis=1)
y_pred_test_labels_2 = y_pred_test_2.argmax(axis=1)

# Menghitung akurasi skor berdasarkan nilai prediksi data latih dan uji
accuracy_train_2 = accuracy_score(y_train, y_pred_train_labels_2)
accuracy_test_2 = accuracy_score(y_test, y_pred_test_labels_2)

# Menampilkan hasil akurasi 
print('\n\nTrain Accuracy: ', accuracy_train_2)
print('Test Accuracy: ', accuracy_test_2)
print('\nClassification Report:\n', classification_report(y_test, y_pred_test_labels_2))

Epoch 1/20
734/734 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.6446 - loss: 0.8381 - val_accuracy: 0.8595 - val_loss: 0.3750 - learning_rate: 0.0010
Epoch 2/20
734/734 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8531 - loss: 0.4117 - val_accuracy: 0.8743 - val_loss: 0.3351 - learning_rate: 0.0010
Epoch 3/20
734/734 ━━━━━━━━━━━━━━━━━━━━ 12s 16ms/step - accuracy: 0.8654 - loss: 0.3603 - val_accuracy: 0.8888 - val_loss: 0.3089 - learning_rate: 0.0010
Epoch 4/20
734/734 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - accuracy: 0.8807 - loss: 0.3183 - val_accuracy: 0.8857 - val_loss: 0.3131 - learning_rate: 0.0010
Epoch 5/20
734/734 ━━━━━━━━━━━━━━━━━━━━ 10s 14ms/step - accuracy: 0.8947 - loss: 0.2881 - val_accuracy: 0.8879 - val_loss: 0.3200 - learning_rate: 0.0010
Epoch 6/20
734/734 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - accuracy: 0.9018 - loss: 0.2715 - val_accuracy: 0.8917 - val_loss: 0.3133 - learning_rate: 0.0010
734/734 ━━━━━━━━━━━━━━━━━━━━ 4s 4ms/step
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms

Model 3

In [17]:
# Model 3 menggunakan arsitektur RNN
Model_3 = Sequential([
    Input(shape=(X_train.shape[1],)),
    Dense(512, activation='relu'),
    Dropout(0.3),
    Dense(300, activation='relu'),
    Dropout(0.3),
    BatchNormalization(),
    Dense(256, activation='relu'),
    Dropout(0.2),
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(3, activation='softmax')
])

# Mengatur optimizer, loss dan metrics yang akan di tampilkan
Model_3.compile(
    optimizer=Adam(learning_rate=0.001),
    loss="sparse_categorical_crossentropy",
    metrics=['accuracy']
)

# Melatih Model RNN
hist_accuracy_3 = Model_3.fit(
    X_train, y_train, epochs=20, batch_size=24, validation_split=0.2, callbacks=[early_stopping, reduce_lr]
)

# Memprediksi model dengan data latih dan uji
y_pred_train_3 = Model_3.predict(X_train)
y_pred_test_3 = Model_3.predict(X_test)

# Konversi prediksi ke label
y_pred_train_labels_3 = y_pred_train_3.argmax(axis=1)
y_pred_test_labels_3 = y_pred_test_3.argmax(axis=1)

# Menghitung akurasi skor berdasarkan nilai prediksi data latih dan uji
accuracy_train_3 = accuracy_score(y_train, y_pred_train_labels_3)
accuracy_test_3 = accuracy_score(y_test, y_pred_test_labels_3)

# Menampilkan hasil akurasi 
print('\n\nTrain Accuracy: ', accuracy_train_3)
print('Test Accuracy: ', accuracy_test_3)
print('\nClassification Report:\n', classification_report(y_test, y_pred_test_labels_3))

Epoch 1/20
783/783 ━━━━━━━━━━━━━━━━━━━━ 18s 14ms/step - accuracy: 0.7512 - loss: 0.6087 - val_accuracy: 0.8557 - val_loss: 0.3829 - learning_rate: 0.0010
Epoch 2/20
783/783 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.8765 - loss: 0.3424 - val_accuracy: 0.8697 - val_loss: 0.3550 - learning_rate: 0.0010
Epoch 3/20
783/783 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - accuracy: 0.9059 - loss: 0.2660 - val_accuracy: 0.8744 - val_loss: 0.3386 - learning_rate: 0.0010
734/734 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step
184/184 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step


Train Accuracy:  0.8822226953374495
Test Accuracy:  0.8647820163487738

Classification Report:
               precision    recall  f1-score   support

           0       0.93      0.92      0.93      4043
           1       0.68      0.10      0.17       383
           2       0.72      0.91      0.80      1446

    accuracy                           0.86      5872
   macro avg       0.78      0.64      0.63      5872
weighted avg       0.86      0.8

Hasil Akhir

In [18]:
result_df = pd.DataFrame({
    'Model': ['Model_1', 'Model_2', 'Model_3'],
    'Accuracy Train': [accuracy_train_1, accuracy_train_2, accuracy_train_3],
    'Accuracy Test': [accuracy_test_1, accuracy_test_2, accuracy_test_3]
})

accuracy_test_all = result_df[['Model', 'Accuracy Test']]
print(accuracy_test_all.sort_values(by='Accuracy Test', ascending=False))

     Model  Accuracy Test
1  Model_2       0.888794
0  Model_1       0.869040
2  Model_3       0.864782


Testing analisis sentimen realtime

In [19]:
# memasukan value kata atau kalimat dari pengguna
data_new = input('Masukan Kalimat Baru: ')
print(data_new)

# pre-processing pada data_new
data_new_filtering = combineFunction(data_new)
data_new_cleaned = toSentence(data_new_filtering)

# Menggunakan objek tfidf yang sudah di-fit dari pelatihan sebelumnya
X_data_new = tf_idf.transform([data_new_cleaned])

# Memprediksi sentimen menggunakan Model_2: karena model ini yang memiliki akurasi paling tinggi
sentimen_predict = Model_2.predict(X_data_new)

# Mendapatkan indeks kelas dengan probabilitas tertinggi
predicted_class = sentimen_predict.argmax(axis=1)[0]

# menampilkan hasil prediksi sesuai dengan score yang didapatkan
if predicted_class == 2:
    print('Sentimen dari kalimat tersebut adalah: POSITIF')
elif predicted_class== 0:
    print('Sentimen dari kalimat tersebut adalah: NEGATIF')
else:
    print('Sentimen dari kalimat tersebut adalah: NETRAL')

aplikasi nya banyak error dan juga bug, kadang juga suka keluar sendiri dan loadingnya lama, aplikasinya buruk
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 759ms/step
Sentimen dari kalimat tersebut adalah: NEGATIF
